In [1]:
import polars as pl

Assessor - Property Tax-Exempt Parcels

- Data Description: Parcels with property tax-exempt status across all of Cook County per tax year, from Tax Year 2022 on, with geographic coordinates and addresses.
- Source: https://datacatalog.cookcountyil.gov/Property-Taxation/Assessor-Property-Tax-Exempt-Parcels/vgzx-68gb/about_data
- Download Date: Oct 16, 2025
- Last Updated: Oct 1, 2025
- Provided By: Cook County Assessor's Office

In [2]:
tax_exempt_raw = pl.read_csv("../data/Assessor_-_Property_Tax-Exempt_Parcels_20251016.csv", 
                            ignore_errors=True,
                            schema_overrides={"pin": pl.Utf8,
                                                "row_id": pl.Utf8,
                                                "owner_num": pl.Utf8,
                                                "township_code": pl.Utf8})

In [3]:
uc_tax_exempt = tax_exempt_raw.filter(
    ((pl.col("owner_name") == "UNIVERSITY OF CHICAGO") | 
    (pl.col("owner_name") == "LAKE PK ASSOC. INC.")) & 
    (pl.col("tax_year") == 2025)
).unique("pin").select(["pin", "tax_year", "township_name", "owner_name", "property_address", "longitude", "latitude"])

In [4]:
uc_tax_exempt_all = tax_exempt_raw.filter(
    ((pl.col("owner_name") == "UNIVERSITY OF CHICAGO") | 
    (pl.col("owner_name") == "LAKE PK ASSOC. INC.")) 
).select(["pin", "tax_year", "township_name", "owner_name", "property_address", "longitude", "latitude"])

In [5]:
tax_exempt_raw.write_parquet("../data/tax_exempt_raw.parquet")

In [6]:
uc_tax_exempt_all.write_parquet("../data/uc_tax_exempt_all.parquet")

Assessor - Commercial Valuation Data

- Data Description: Commercial valuation data collected and maintained by the Cook County Assessor's Office, from 2021 to present. The office uses this data primarily for valuation and reporting. This dataset consolidates the individual Excel workbooks available on the Assessor's website into a single shared format. Properties are valued using similar valuation methods within each model group, per township, per year (in the year the township is reassessed). This dataset has been cleaned minimally, only enough to fit the source Excel workbooks together - because models are updated for each township in the year it is reassessed, users should expect inconsistencies within columns across time and townships.

- Source: https://datacatalog.cookcountyil.gov/Property-Taxation/Assessor-Commercial-Valuation-Data/csik-bsws/about_data
- Download Date: Oct 16, 2025
- Last Updated: April 10, 2025
- Provided By: Cook County Assessor's Office

In [7]:
com_val_raw = pl.read_csv("../data/Assessor_-_Commercial_Valuation_Data_20251016.csv", 
                            ignore_errors=True,
                            schema_overrides={"keypin": pl.Utf8,
                                                "pins": pl.Utf8,
                                                "owner_num": pl.Utf8,
                                                "township_code": pl.Utf8})

In [8]:
# dedupe so that we only have the latest market value

com_val = (com_val_raw
           .with_columns(
                pl.col("keypin").str.replace_all("-",""),
                pl.col("pins").str.replace_all("-",""),
                pl.col("finalmarketvalue").str.replace_all(r"\$","").str.replace_all(",","")
                )
                .sort("year", descending=True)
                .unique("keypin", keep='first', maintain_order=True)
)

Assessor - Assessed Values

- Data Description: Land, building, and total assessed values for all Cook County parcels, from 1999 to present. The Assessor's Office uses these values for reporting, evaluating assessment performance over time, and research.

- Source: https://datacatalog.cookcountyil.gov/Property-Taxation/Assessor-Assessed-Values/uzyt-m557/about_data
- Download Date: Oct 16, 2025
- Last Updated: Oct 1, 2025
- Provided By: Cook County Assessor's Office

In [9]:
ass_val_raw = pl.read_csv("../data/Assessor_-_Assessed_Values_20251016.csv", 
                            ignore_errors=True,
                            schema_overrides={"pin": pl.Utf8,
                                                "row_id": pl.Utf8,
                                                "neighborhood_code": pl.Utf8,
                                                "township_code": pl.Utf8})

In [10]:
ass_val = (ass_val_raw
           .select(["pin", "tax_year", "mailed_tot", "certified_tot", "board_tot"])
           .with_columns(pl.col("mailed_tot").str.replace_all(r"\$","").str.replace_all(",",""),
                         pl.col("certified_tot").str.replace_all(r"\$","").str.replace_all(",",""),
                         pl.col("board_tot").str.replace_all(r"\$","").str.replace_all(",",""))
)

Assessor - Parcel Sales

- Data Description: Parcel sales for real property in Cook County, from 1999 to present. The Assessor's Office uses this data in its modeling to estimate the fair market value of unsold properties.

- Source: https://datacatalog.cookcountyil.gov/Property-Taxation/Assessor-Parcel-Sales/wvhk-k5uv/about_data
- Download Date: Oct 16, 2025
- Last Updated: Oct 1, 2025
- Provided By: Cook County Assessor's Office

In [ ]:
sales_raw = pl.read_csv("../data/Assessor_-_Parcel_Sales_20251016.csv", 
                            ignore_errors=True,
                            schema_overrides={"pin": pl.Utf8,
                                                "row_id": pl.Utf8,
                                                "neighborhood_code": pl.Utf8,
                                                "township_code": pl.Utf8})

In [30]:
# dedupe so that we only have the latest sales value
sales = sales_raw.with_columns(pl.col("sale_price").str.replace_all(r"\$","").str.replace_all(",","")).with_columns(pl.col("sale_price").cast(pl.Float64))


sales_dedupe = (sales
            .sort("year", descending=True)
            .unique("pin", keep='first', maintain_order=True)
).select(["pin", "sale_date", "sale_price", "sale_seller_name", "sale_buyer_name"])

In [ ]:
uc_purchases = sales_raw.filter(
    (pl.col("sale_buyer_name") == "UNIVERSITY OF CHICAGO") |
    (pl.col("sale_buyer_name") == "THE UNIVERSITY OF CHICAGO") |
    (pl.col("sale_buyer_name") == "LAKE PARK ASSOCIATES  INC") |
    (pl.col("sale_buyer_name") == "LAKE PARK ASSOCIATES, INC") |
    (pl.col("sale_buyer_name") == "LAKE PARK ASSOCIATES INC.") |
    (pl.col("sale_buyer_name") == "LAKE PARK ASSOC INC") |
    (pl.col("sale_buyer_name") == "LAKE PARK ASSOCIATES INC") |
    (pl.col("sale_buyer_name") == "LAKE PARK ASSOCIATES, INC.") 
)

In [31]:
sales.write_parquet("../data/sales.parquet")

In [14]:
uc_purchases.write_parquet("../data/uc_purchases.parquet")

Assessor [Archived 05-11-2022] - Property Locations

- Data Description: This data set includes property locations and attached spatial data for all Cook County parcels. Spatial matching was based on parcel centroids. Older properties may be missing latitude and longitude data because they are not contained in the most recent parcel shape layer. Brand new properties may be missing a mailing/property address because the postal service has not yet assigned the property an address. Data attached to each PIN is the most recent available. For example, Census tract data is drawn from the 2014-2018 5-year American Community Survey.

- Source: https://datacatalog.cookcountyil.gov/Property-Taxation/Assessor-Archived-05-11-2022-Property-Locations/c49d-89sn/about_data
- Download Date: Oct 18, 2025
- Last Updated: Jan 26, 2021
- Provided By: Cook County Assessor's Office

In [74]:
loc_raw = pl.read_csv("../data/Assessor_Archived_05-11-2022_Property_Locations_20251018.csv", 
                            ignore_errors=True,
                            schema_overrides={"pin": pl.Utf8,
                                              "township": pl.Utf8})

loc = loc_raw.with_columns(
    (pl.col("tract_pop").str.replace_all(",","")).cast(pl.Int64),
    (pl.col("tract_midincome").str.replace_all(",","")).cast(pl.Int64),
)

### Merge all datasets

In [76]:
merge = (uc_tax_exempt
         .join(sales_dedupe, on="pin", how="left")
         .join(ass_val, on=["pin", "tax_year"], how='left')
         .join(com_val, left_on="pin", right_on="keypin", how='left')
         .join(loc, on="pin", how="left")
)

In [77]:
clean_combined = merge.with_columns(
    pl.col("sale_date").str.strptime(pl.Date, format="%B %d, %Y"),
    pl.col("sale_price").cast(pl.Float64),
    pl.col("mailed_tot").cast(pl.Float64),
    pl.col("certified_tot").cast(pl.Float64),
    pl.col("board_tot").cast(pl.Float64),
    pl.col("finalmarketvalue").cast(pl.Float64)
)

In [78]:
clean_combined.write_parquet("../data/clean_combined.parquet")

In [79]:
# Not UChicago-specific

uni_tax_exempt = (
    tax_exempt_raw
    .with_columns(
        pl.when(pl.col("owner_name") == "DE PAUL UNIVERSITY")
        .then(pl.lit("DEPAUL UNIVERSITY"))
        .when(pl.col("owner_name") == "UNIVERSITY OF ILLINOIS FOUNDATION")
        .then(pl.lit("UNIVERSITY OF ILLINOIS"))
        .when(pl.col("owner_name") == "LAKE PK ASSOC. INC.")
        .then(pl.lit("UNIVERSITY OF CHICAGO"))         
        .when(pl.col("owner_name") == "LOYOLA UNIV MED CENTER")
        .then(pl.lit("LOYOLA UNIV OF CHGO")) 
        .when(pl.col("owner_name") == "JESUIT COM/LOYOLA UNIV")
        .then(pl.lit("LOYOLA UNIV OF CHGO"))     
        .otherwise(pl.col("owner_name"))
        .alias("owner_name")  
    ).filter(
        (pl.col("property_city") == "CHICAGO") &
        (
            (pl.col("owner_name").str.contains("UNIVERSITY OF CHICAGO")) |
            (pl.col("owner_name").str.contains("UNIVERSITY OF ILLINOIS")) | 
            (pl.col("owner_name").str.contains("LOYOLA UNIV OF CHGO")) | 
            (pl.col("owner_name").str.contains("DEPAUL UNIVERSITY")) 
    )
))

merge_all = (uni_tax_exempt
         .join(sales_dedupe, on="pin", how="left")
         .join(ass_val, on=["pin", "tax_year"], how='left')
         .join(com_val, left_on="pin", right_on="keypin", how='left')
         .join(loc, on="pin", how="left")
)

In [80]:
clean_combined_all = merge_all.with_columns(
    pl.col("sale_date").str.strptime(pl.Date, format="%B %d, %Y"),
    pl.col("sale_price").cast(pl.Float64),
    pl.col("mailed_tot").cast(pl.Float64),
    pl.col("certified_tot").cast(pl.Float64),
    pl.col("board_tot").cast(pl.Float64),
    pl.col("finalmarketvalue").cast(pl.Float64)
)

clean_combined_all.write_parquet("../data/clean_combined_all.parquet")